In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import utils as u
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
# Load track info
tracks = [
    {"raga": "thodi", "piece": "Karuna Nidhi Illalo"},
    {"raga": "thodi", "piece": "Koluvamaregatha"},
    {"raga": "thodi", "piece": "Munnu Ravana"},
    {"raga": "thodi", "piece": "Thappi Bratikipova"},
    {"raga": "kalyani", "piece": "Kailasapathe"},
    {"raga": "kalyani", "piece": "Sundari Nee Divya"},
    {"raga": "kalyani", "piece": "Vandalum"}
]
# Initialize an empty list to store DataFrames
dfs = []

# Load data
def append_df(dfs, raga, piece, section, instrument, file_path):
    df = pd.read_csv(file_path)
    df["raga"] = raga
    df["piece"] = piece
    df["section"] = section
    df["instrument"] = instrument
    dfs.append(df)

for track in tracks:
    raga = track["raga"]
    piece = track["piece"]
    if piece == "Munnu Ravana" or piece == "Vandalum":
        vifile = f"{raga}/{piece}/{piece}.multitrack-violin-peaks.csv"
        append_df(dfs, raga, piece, "main", "violin", vifile)
        
        vofile = f"{raga}/{piece}/{piece}.multitrack-vocal-peaks.csv"
        append_df(dfs, raga, piece, "main", "vocal", vofile)
    else:
        viafile = f"{raga}/{piece}/{piece}.multitrack-violin-alaapana-peaks.csv"
        append_df(dfs, raga, piece, "alaapana", "violin", viafile)
        
        vimfile = f"{raga}/{piece}/{piece}.multitrack-violin-main-peaks.csv"
        append_df(dfs, raga, piece, "main", "violin", vimfile)
        
        visfile = f"{raga}/{piece}/{piece}.multitrack-violin-solo-peaks.csv"
        append_df(dfs, raga, piece, "solo", "violin", visfile)
        
        voafile = f"{raga}/{piece}/{piece}.multitrack-vocal-alaapana-peaks.csv"
        append_df(dfs, raga, piece, "alaapana", "vocal", voafile)
        
        vomfile = f"{raga}/{piece}/{piece}.multitrack-vocal-main-peaks.csv"
        append_df(dfs, raga, piece, "main", "vocal", vomfile)

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Move piece, section, and instrument columns to the front
cols = combined_df.columns.tolist()
cols = cols[-4:] + cols[:-4]
combined_df = combined_df[cols]
        

In [14]:
swara_dict = {
    "s": 0,
    "R1": 100,
    "R2": 200,
    "G1": 300,
    "G3": 400,
    "M1": 500,
    "M2": 600,
    "P": 700,
    "D1": 800,
    "D2": 900,
    "N2": 1000,
    "N3": 1100,
    "S": 1200
}

def find_closest_swara(mean_cents):
    closest_swara = min(swara_dict, key=lambda k: abs(swara_dict[k] - mean_cents))
    return closest_swara

# Find closest swara for each peak
combined_df['swara'] = combined_df['Mean (cents)'].apply(find_closest_swara)
# Move swara before mean cents
cols = combined_df.columns.tolist()
swara_col = cols.pop(cols.index('swara'))
cols.insert(cols.index('Mean (cents)'), swara_col)
combined_df = combined_df[cols]
combined_df

,raga,piece,section,instrument,swara,Mean (cents),Amplitude (normalized),Standard Deviation (cents),Offset (normalized),Skewness
0,thodi,Karuna Nidhi Illalo,alaapana,violin,s,5,0.003755,1.651280,-0.048700,0.409570
1,thodi,Karuna Nidhi Illalo,alaapana,violin,R1,90,0.919094,333.207508,1.098036,-0.000030
2,thodi,Karuna Nidhi Illalo,alaapana,violin,R2,206,0.007385,14.628238,-0.007141,-0.511851
3,thodi,Karuna Nidhi Illalo,alaapana,violin,M1,505,0.003427,39.570964,-0.003914,-5.126788
4,thodi,Karuna Nidhi Illalo,alaapana,violin,P,702,0.004226,25.983535,-0.004238,3.755487
...,...,...,...,...,...,...,...,...,...,...
188,kalyani,Vandalum,main,vocal,M2,607,0.044728,19.448756,0.090064,1.476283
189,kalyani,Vandalum,main,vocal,P,691,0.030482,30.254939,0.097354,-4.408457
190,kalyani,Vandalum,main,vocal,D2,903,0.041001,31.301560,0.059275,3.350956
191,kalyani,Vandalum,main,vocal,N3,1111,0.142906,17.855094,0.139149,1.126694
